In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기

#padding 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pprint as pp
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
def read_data_from_file(file_path):
    sentence_file = open(file_path, 'r')
    sentences = []
    while True:
        line = sentence_file.readline()
        if not line:
            break;
        sentences.append(line.replace('\n',''))
    sentence_file.close()
    
    rev_vocab_dic = list(set(' '.join(sentences)))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return sentences, vocab_dic, rev_vocab_dic

In [3]:
chosung_sentences, chosung_vocab_dic, chosung_rev_vocab_dic = read_data_from_file('data/dev/namuwiki_chosung_dev.txt')
target_sentences, target_vocab_dic, target_rev_vocab_dic =  read_data_from_file('data/dev/namuwiki_sentences_dev.txt')

In [4]:
# sentences = ["그는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다.",
#             "김씨가 아버지에게 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다."]
# char_set = list(set(' '.join(sentences)))
# char_dic = {w: i for i, w in enumerate(char_set)}
# print(char_dic)

In [5]:
# chosung_sentences = ["ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄴㄷㅎㄷ.",
#                     "ㄱㅆㄱ ㅇㅂㅈㅇㄱ ㅂㄴㄴ ㅍㅈㄹ ㅇㄱ ㅁㄷ ㄷㄹ ㅌㅈㅎㄹ ㅎㅈ ㅁ ㄷㅌㄹㅇ ㅈㄹㅇㅅ ㅇㅇㄴ ㄱㅆㄹ ㄸㄹㄱ ㅇㅇㅈㅁ ㅇㄹㅎㄷ."]
# chosung_set = list(set(' '.join(chosung_sentences)))
# chosung_dic = {w: i for i, w in enumerate(chosung_set)}
# print(chosung_dic)

In [6]:
data_dim = len(target_vocab_dic)
hidden_size = len(target_vocab_dic)
num_classes = len(target_vocab_dic)
embedding_size = len(target_vocab_dic)
sequence_length = 22  # Any arbitrary number
learning_rate = 0.05

In [7]:
dataX = []
dataY = []
for i in range(len(chosung_sentences)):
    chosung_sentence = chosung_sentences[i][:sequence_length]
    target_sentence = target_sentences[i][:sequence_length]
    print(i, chosung_sentence, '->', target_sentence)
    x = [chosung_vocab_dic[c] for c in chosung_sentence]  # x str to index
    y = [target_vocab_dic[c] for c in target_sentence]  # y str to index
    dataX.append(x)
    dataY.append(y)

0  ㄱㅌ ㄱㅌ ㅈㅂㄱ ㅇㅇㅁ ㅅㅇㅎ ㅈㅅㅇ ->  기타 기타 정보가 있으면 삽입해 주세요
1  ㄱㅌ ㄱㅌ ㅈㅂㄱ ㅇㅇㅁ ㅅㅇㅎ ㅈㅅㅇ ->  기타 기타 정보가 있으면 삽입해 주세요
2 ㅇㄹ <english> ㅈㅈㅅㄷㄱ ㅎㄹㅎ -> 여러 <english> 제조사들과 협력하
3 ㄱㅈ ㄱㄱㄷㄱ ㅊㅂㅎ ㄷㄴ ㅌㅈㅇ ㅇㄴㄷ -> 기존 기기들과 차별화 되는 특징이 있는데
4 <number>ㄴ ㅎㅈ ㅇㄸ ㅈㅈㅅㅇ ㅈ -> <number>년 현재 어떤 제조사의 제
5  ㄱㅌ ㄱㅌ ㅈㅂㄱ ㅇㅇㅁ ㅅㅇㅎ ㅈㅅㅇ ->  기타 기타 정보가 있으면 삽입해 주세요
6  ㄱㅌ ㄱㅌ ㅈㅂㄱ ㅇㅇㅁ ㅅㅇㅎ ㅈㅅㅇ ->  기타 기타 정보가 있으면 삽입해 주세요
7  ㄱㅌ ㄱㅌ ㅈㅂㄱ ㅇㅇㅁ ㅅㅇㅎ ㅈㅅㅇ ->  기타 기타 정보가 있으면 삽입해 주세요
8 <english>ㄱㄹ ㅅㅅㅇ ㅈㄱ ㅎㅇ  -> <english>그룹 소속의 중견 해운 
9 ㅂㅋㅅ ㅂㅇㅇㅅㄴ ㄱㄴ <number>ㅇ -> 벌크선 분야에서는 국내 <number>위


In [8]:
# test_sentence = "아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 그는 이날 기념식에서 낭독했다."
# test_chosung_sentence = "ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㄴㄷㅎㄷ."

# test_dataX = []
# test_dataY = []
# for i in range(0, len(test_sentence) - sequence_length):
#     x_str = test_sentence[i:i + sequence_length]
#     y_str = test_chosung_sentence[i:i + sequence_length]
#     print(i, x_str, '->', y_str)

#     x = [char_dic[c] for c in x_str]  # x str to index
#     y = [chosung_dic[c] for c in y_str]  # y str to index

#     test_dataX.append(x)
#     test_dataY.append(y)

In [9]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
# X_one_hot = tf.one_hot(X, embedding_size)
# print(X_one_hot)  # check out the shape

In [10]:
print(num_classes)
embeddings = tf.Variable(
    tf.random_uniform([num_classes, 10], -1.0, 1.0), trainable=False)
X_one_hot = tf.nn.embedding_lookup(embeddings, X)
print(X_one_hot)

147
Tensor("embedding_lookup:0", shape=(?, 22, 10), dtype=float32)


In [11]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    if i % 10 == 1:
        print('step:',i,' loss:',l)
        embds, results = sess.run([embeddings,outputs], feed_dict={X: dataX})
#         print(embds)
        for sentence_result in results:
            index = np.argmax(sentence_result, axis=1)
            print(''.join([target_rev_vocab_dic[t] for t in index]))
        print('----------')


step: 1  loss: 4.16183
r미으으으있e재h세으으으e재재재세으으으으
r미으으으있e재h세으으으e재재재세으으으으
참면으으e재h해으으e재h으으으으으e재h세
h과으으으으e재재h으으으으e재h세으으으으
공면으으으e재h으으으으e재h으으으으으e재
r미으으으있e재h세으으으e재재재세으으으으
r미으으으있e재h세으으으e재재재세으으으으
r미으으으있e재h세으으으e재재재세으으으으
공면으으으e재재h으으으e재h세으으으으e재
l공면으으으재재재세으으으재재h나사으으재h
step: 11  loss: 2.71208
 기타 기타 타   타    입  요 요
 기타 기타 타   타    입  요 요
여러  제  사   사 사  사  요요 
기기 타    요   요  요 요 요  
nnnnnn  어  의 제  제  제 사
 기타 기타 타   타    입  요 요
 기타 기타 타   타    입  요 요
 기타 기타 타   타    입  요 요
nnnnbih       제 제  사사 
 기   타  요 요  요      요입
step: 21  loss: 0.776901
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
여러 <engggsh> 제력사협하 제력하
기기 기기기들 있별특 주는 특별이 특는는
<number>년 현재 어떤 제운사의 제
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
<english>그룹 소속의 제견 제운 
벌크선 분야에 e <e e에en<b en
step: 31  loss: 0.0751704
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
여러 <english> 제조사들과 협력하
기존 기기들과 차별화 되는 특징이 있는데
<number>년 현재 어떤 제조사의 제
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
 기타 기타 정보가 있으면 삽입해 주세요
<english>그룹 소속의 제견 해운 
벌크선 분

In [18]:
print(len(results)) #batch size
print(len(results[0])) #sequence length
print(len(results[0][0])) #vocab_size

print(batch_size)
print(sequence_length)
print(len(target_vocab_dic))


10
22
147
10
22
147


In [ ]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)